## Importing NIH Dataset (ZIP Format)

In [1]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

--2020-03-04 03:38:22--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  16.5MB/s    in 8.6s    

2020-03-04 03:38:32 (39.4 MB/s) - ‘cell_images.zip’ saved [353452851]

Extracting images...
Done!


## Unzip Images, Resize, and Store in NumPy Arrays

In [2]:
# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([13779,128,128,3])
Uninfected = np.empty([13779,128,128,3])

# Resize and load parasitized images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

     |████████████████████████████████| 28.2 MB 3.4 MB/s eta 0:00:01
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                    
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                m
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease          
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done0m0m                  
Building dependency tree       
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree  

## Generate Cross-Validation Indices for Training and Testing Sets

In [3]:
# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], 13779, axis=0)
UninfectedLabels = np.repeat([[1,0]], 13779, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
Index1, Index2, Index3, Index4, Index5 = CVIndices[:5512], CVIndices[5512:11024], CVIndices[11024:16536], CVIndices[16536:22048], CVIndices[22048:]

## Create List of Classification Layer Hyperparameters

In [4]:
# Import relevant neural network architecture packages 
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


## Train Model and Save Results as CSV Files
Noise at all levels

In [5]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, GaussianNoise
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# Create empty lists to store results
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
All_FPR = []
All_TPR = []
All_thresholds = []
All_AUC = []

for i in range(5):

    # Create the appropriate training and testing sets
    if i == 0:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
        TestImages = Dataset[Index5,:]
        TestLabels = Labels[Index5,:]
    elif i == 1:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index4,:]
        TestLabels = Labels[Index4,:]
    elif i == 2:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index3,:]
        TestLabels = Labels[Index3,:]
    elif i == 3:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index2,:]
        TestLabels = Labels[Index2,:]
    else:
        TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index1,:]
        TestLabels = Labels[Index1,:]

    base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

    # Store the fully connected layers
    Input = base_model.layers[0]
    ConvBlock1_1 = base_model.layers[1]
    ConvBlock1_2 = base_model.layers[2]
    ConvBlock1_Pool = base_model.layers[3]
    ConvBlock2_1 = base_model.layers[4]
    ConvBlock2_2 = base_model.layers[5]
    ConvBlock2_Pool = base_model.layers[6]
    ConvBlock3_1 = base_model.layers[7]
    ConvBlock3_2 = base_model.layers[8]
    ConvBlock3_3 = base_model.layers[9]
    ConvBlock3_Pool = base_model.layers[10]
    ConvBlock4_1 = base_model.layers[11]
    ConvBlock4_2 = base_model.layers[12]
    ConvBlock4_3 = base_model.layers[13]
    ConvBlock4_Pool = base_model.layers[14]
    ConvBlock5_1 = base_model.layers[15]
    ConvBlock5_2 = base_model.layers[16]
    ConvBlock5_3 = base_model.layers[17]
    ConvBlock5_Pool = base_model.layers[18]

    #NewLayer = ConvBlock5_3(ConvBlock5_2)


    # Reconstructing neural network architecture w/ batch normalization
    x = Input.output
    x = ConvBlock1_1(x)
    x = ConvBlock1_2(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock1_Pool(x)
    x = ConvBlock2_1(x)
    x = ConvBlock2_2(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock2_Pool(x)
    x = ConvBlock3_1(x)
    x = ConvBlock3_2(x)
    x = ConvBlock3_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock3_Pool(x)
    x = ConvBlock4_1(x)
    x = ConvBlock4_2(x)
    x = ConvBlock4_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock4_Pool(x)
    x = ConvBlock5_1(x)
    x = ConvBlock5_2(x)
    x = ConvBlock5_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock5_Pool(x)

    ### INSERT REST OF LAYERS HERE ###
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(input = base_model.input, output = predictions)
    sgd = optimizers.SGD(learning_rate=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

    # Train model and evaluate performance
    print('We are now training cross-validation set #',i+1)
    Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

    # Display and store performance results
    Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
    Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
    Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
    Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

    print('Training Loss:',Results.history['loss'])
    print('Training Accuracy:',Results.history['accuracy'])
    print('Validation Loss:',Results.history['val_loss'])
    print('Validation Accuracy:',Results.history['val_accuracy'])

    TrainLoss.append(Results.history['loss'])
    TrainAcc.append(Results.history['accuracy'])
    TestLoss.append(Results.history['val_loss'])
    TestAcc.append(Results.history['val_accuracy'])
    print('')


    # Predict values for test set
    Probabilities = model.predict(TestImages)

    # Calculate data for ROC curve
    FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
    All_FPR.append(FPR)
    All_TPR.append(TPR)
    All_thresholds.append(thresholds)

# Save and export as CSV files
with open("AllNoise_TrainLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainLoss)
with open("AllNoise_TrainAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainAcc)
with open("AllNoise_TestLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestLoss)
with open("AllNoise_TestAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestAcc)
with open("AllNoise_FPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_FPR)
with open("AllNoise_TPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_TPR)
with open("AllNoise_Thresholds.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_thresholds)

     |████████████████████████████████| 7.1 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 81.4 MB/s eta 0:00:01
58892288/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 1
Train on 22048 samples, validate on 5510 samples
Epoch 1/75
22048/22048 [==============================] - 47s 2ms/step - loss: 0.6985 - accuracy: 0.7816 - val_loss: 0.1653 - val_accuracy: 0.9450
Epoch 2/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.2216 - accuracy: 0.9204 - val_loss: 0.1428 - val_accuracy: 0.9550
Epoch 3/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1925 - accuracy: 0.9373 - val_loss: 0.1373 - val_accuracy: 0.9568
Epoch 4/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1699 - accuracy: 0.9443 - val_loss: 0.1329 - val_accuracy: 0.9579
Epoch 5/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1556 - accuracy: 0.9490 - val_loss: 0.1278 - val_accuracy: 0.9592
Epoch 6/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1517 - accuracy: 0.9510 - val_loss: 0.1241 - val_accuracy: 0.9610
Epoch 7/75
22048/22048 [

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22048/22048 [==============================] - 40s 2ms/step - loss: 0.0802 - accuracy: 0.9704 - val_loss: 0.1069 - val_accuracy: 0.9664
Epoch 49/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0798 - accuracy: 0.9702 - val_loss: 0.1079 - val_accuracy: 0.9661
Epoch 50/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0780 - accuracy: 0.9707 - val_loss: 0.1022 - val_accuracy: 0.9673
Epoch 51/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0777 - accuracy: 0.9721 - val_loss: 0.1008 - val_accuracy: 0.9670
Epoch 52/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0754 - accuracy: 0.9715 - val_loss: 0.1061 - val_accuracy: 0.9661
Epoch 53/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0764 - accuracy: 0.9728 - val_loss: 0.1054 - val_accuracy: 0.9659
Epoch 54/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.0738 - accuracy: 0.9721 - val_loss: 0.1036

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22046/22046 [==============================] - 40s 2ms/step - loss: 0.0699 - accuracy: 0.9730 - val_loss: 0.1220 - val_accuracy: 0.9639
Epoch 54/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0665 - accuracy: 0.9761 - val_loss: 0.1200 - val_accuracy: 0.9624
Epoch 55/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0658 - accuracy: 0.9758 - val_loss: 0.1235 - val_accuracy: 0.9628
Epoch 56/75
 4416/22046 [=====>........................] - ETA: 29s - loss: 0.0666 - accuracy: 0.9740

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22046/22046 [==============================] - 40s 2ms/step - loss: 0.0582 - accuracy: 0.9780 - val_loss: 0.1221 - val_accuracy: 0.9614
Epoch 65/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0576 - accuracy: 0.9783 - val_loss: 0.1284 - val_accuracy: 0.9610
Epoch 66/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0578 - accuracy: 0.9789 - val_loss: 0.1366 - val_accuracy: 0.9628
Epoch 67/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0565 - accuracy: 0.9799 - val_loss: 0.1233 - val_accuracy: 0.9621
Epoch 68/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0553 - accuracy: 0.9788 - val_loss: 0.1344 - val_accuracy: 0.9630
Epoch 69/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0527 - accuracy: 0.9804 - val_loss: 0.1326 - val_accuracy: 0.9621
Epoch 70/75
22046/22046 [==============================] - 40s 2ms/step - loss: 0.0531 - accuracy: 0.9801 - val_loss: 0.1288

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Dense Noise Only

In [6]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, GaussianNoise
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# Create empty lists to store results
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
All_FPR = []
All_TPR = []
All_thresholds = []
All_AUC = []

for i in range(5):

    # Create the appropriate training and testing sets
    if i == 0:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
        TestImages = Dataset[Index5,:]
        TestLabels = Labels[Index5,:]
    elif i == 1:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index4,:]
        TestLabels = Labels[Index4,:]
    elif i == 2:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index3,:]
        TestLabels = Labels[Index3,:]
    elif i == 3:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index2,:]
        TestLabels = Labels[Index2,:]
    else:
        TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index1,:]
        TestLabels = Labels[Index1,:]

    base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

    # Store the fully connected layers
    Input = base_model.layers[0]
    ConvBlock1_1 = base_model.layers[1]
    ConvBlock1_2 = base_model.layers[2]
    ConvBlock1_Pool = base_model.layers[3]
    ConvBlock2_1 = base_model.layers[4]
    ConvBlock2_2 = base_model.layers[5]
    ConvBlock2_Pool = base_model.layers[6]
    ConvBlock3_1 = base_model.layers[7]
    ConvBlock3_2 = base_model.layers[8]
    ConvBlock3_3 = base_model.layers[9]
    ConvBlock3_Pool = base_model.layers[10]
    ConvBlock4_1 = base_model.layers[11]
    ConvBlock4_2 = base_model.layers[12]
    ConvBlock4_3 = base_model.layers[13]
    ConvBlock4_Pool = base_model.layers[14]
    ConvBlock5_1 = base_model.layers[15]
    ConvBlock5_2 = base_model.layers[16]
    ConvBlock5_3 = base_model.layers[17]
    ConvBlock5_Pool = base_model.layers[18]

    #NewLayer = ConvBlock5_3(ConvBlock5_2)


    # Reconstructing neural network architecture w/ batch normalization
    x = Input.output
    x = ConvBlock1_1(x)
    x = ConvBlock1_2(x)
    #x = GaussianNoise(0.1)(x)
    x = ConvBlock1_Pool(x)
    x = ConvBlock2_1(x)
    x = ConvBlock2_2(x)
    #x = GaussianNoise(0.1)(x)
    x = ConvBlock2_Pool(x)
    x = ConvBlock3_1(x)
    x = ConvBlock3_2(x)
    x = ConvBlock3_3(x)
    #x = GaussianNoise(0.1)(x)
    x = ConvBlock3_Pool(x)
    x = ConvBlock4_1(x)
    x = ConvBlock4_2(x)
    x = ConvBlock4_3(x)
    #x = GaussianNoise(0.1)(x)
    x = ConvBlock4_Pool(x)
    x = ConvBlock5_1(x)
    x = ConvBlock5_2(x)
    x = ConvBlock5_3(x)
    #x = GaussianNoise(0.1)(x)
    x = ConvBlock5_Pool(x)

    ### INSERT REST OF LAYERS HERE ###
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(input = base_model.input, output = predictions)
    sgd = optimizers.SGD(learning_rate=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

    # Train model and evaluate performance
    print('We are now training cross-validation set #',i+1)
    Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

    # Display and store performance results
    Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
    Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
    Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
    Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

    print('Training Loss:',Results.history['loss'])
    print('Training Accuracy:',Results.history['accuracy'])
    print('Validation Loss:',Results.history['val_loss'])
    print('Validation Accuracy:',Results.history['val_accuracy'])

    TrainLoss.append(Results.history['loss'])
    TrainAcc.append(Results.history['accuracy'])
    TestLoss.append(Results.history['val_loss'])
    TestAcc.append(Results.history['val_accuracy'])
    print('')


    # Predict values for test set
    Probabilities = model.predict(TestImages)

    # Calculate data for ROC curve
    FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
    All_FPR.append(FPR)
    All_TPR.append(TPR)
    All_thresholds.append(thresholds)

# Save and export as CSV files
with open("DenseNoise_TrainLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainLoss)
with open("DenseNoise_TrainAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainAcc)
with open("DenseNoise_TestLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestLoss)
with open("DenseNoise_TestAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestAcc)
with open("DenseNoise_FPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_FPR)
with open("DenseNoise_TPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_TPR)
with open("DenseNoise_Thresholds.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_thresholds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 1
Train on 22048 samples, validate on 5510 samples
Epoch 1/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.7060 - accuracy: 0.7686 - val_loss: 0.2129 - val_accuracy: 0.9238
Epoch 2/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.2552 - accuracy: 0.9019 - val_loss: 0.1575 - val_accuracy: 0.9465
Epoch 3/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1967 - accuracy: 0.9307 - val_loss: 0.1432 - val_accuracy: 0.9557
Epoch 4/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1702 - accuracy: 0.9413 - val_loss: 0.1375 - val_accuracy: 0.9568
Epoch 5/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1595 - accuracy: 0.9443 - val_loss: 0.1333 - val_accuracy: 0.9588
Epoch 6/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1530 - accuracy: 0.9472 - val_loss: 0.1284 - val_accuracy: 0.9572
Epoch 7/75
22048/22048 [

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22048/22048 [==============================] - 38s 2ms/step - loss: 0.1103 - accuracy: 0.9605 - val_loss: 0.1118 - val_accuracy: 0.9626
Epoch 20/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1077 - accuracy: 0.9617 - val_loss: 0.1115 - val_accuracy: 0.9630
Epoch 21/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1085 - accuracy: 0.9611 - val_loss: 0.1150 - val_accuracy: 0.9637
Epoch 22/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1063 - accuracy: 0.9608 - val_loss: 0.1093 - val_accuracy: 0.9624 - ac - ETA: 0s - loss: 0.1062 - accuracy
Epoch 23/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1051 - accuracy: 0.9623 - val_loss: 0.1081 - val_accuracy: 0.9635
Epoch 24/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1042 - accuracy: 0.9626 - val_loss: 0.1088 - val_accuracy: 0.9644
Epoch 25/75
22048/22048 [==============================] - 38s 2ms/step - loss: 0.1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22046/22046 [==============================] - 38s 2ms/step - loss: 0.0790 - accuracy: 0.9721 - val_loss: 0.1185 - val_accuracy: 0.9626
Epoch 45/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0769 - accuracy: 0.9717 - val_loss: 0.1165 - val_accuracy: 0.9615
Epoch 46/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0773 - accuracy: 0.9717 - val_loss: 0.1177 - val_accuracy: 0.9623
Epoch 47/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0732 - accuracy: 0.9728 - val_loss: 0.1221 - val_accuracy: 0.9617
Epoch 48/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0736 - accuracy: 0.9724 - val_loss: 0.1157 - val_accuracy: 0.9639
Epoch 49/75
22046/22046 [==============================] - 37s 2ms/step - loss: 0.0739 - accuracy: 0.9726 - val_loss: 0.1156 - val_accuracy: 0.9644
Epoch 50/75
22046/22046 [==============================] - 37s 2ms/step - loss: 0.0692 - accuracy: 0.9750 - val_loss: 0.1186

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22046/22046 [==============================] - 38s 2ms/step - loss: 0.0594 - accuracy: 0.9780 - val_loss: 0.1267 - val_accuracy: 0.9619
Epoch 64/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0583 - accuracy: 0.9779 - val_loss: 0.1265 - val_accuracy: 0.9639
Epoch 65/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0576 - accuracy: 0.9780 - val_loss: 0.1275 - val_accuracy: 0.9626
Epoch 66/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0576 - accuracy: 0.9785 - val_loss: 0.1206 - val_accuracy: 0.9632
Epoch 67/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.0566 - accuracy: 0.9788 - val_loss: 0.1288 - val_accuracy: 0.9630s - loss: 0.0576 - accuracy:  - ETA: 4s - loss: 0.0576 -  - ETA: 3s - loss: 0.0577 - accuracy:  - ETA: 3s - - ETA: 1s - loss: 0.0570 - accura - ETA: 1s - loss: 0.0 - ETA: 0s - loss: 0.0566 - accuracy: 0.
Epoch 68/75
22046/22046 [==============================] - 38s 2ms/st

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Convolution Noise Only

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, GaussianNoise
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# Create empty lists to store results
TrainLoss = []
TrainAcc = []
TestLoss = []
TestAcc = []
All_FPR = []
All_TPR = []
All_thresholds = []
All_AUC = []

for i in range(5):

    # Create the appropriate training and testing sets
    if i == 0:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
        TestImages = Dataset[Index5,:]
        TestLabels = Labels[Index5,:]
    elif i == 1:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index4,:]
        TestLabels = Labels[Index4,:]
    elif i == 2:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index3,:]
        TestLabels = Labels[Index3,:]
    elif i == 3:
        TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index2,:]
        TestLabels = Labels[Index2,:]
    else:
        TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
        TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
        TestImages = Dataset[Index1,:]
        TestLabels = Labels[Index1,:]

    base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

    # Store the fully connected layers
    Input = base_model.layers[0]
    ConvBlock1_1 = base_model.layers[1]
    ConvBlock1_2 = base_model.layers[2]
    ConvBlock1_Pool = base_model.layers[3]
    ConvBlock2_1 = base_model.layers[4]
    ConvBlock2_2 = base_model.layers[5]
    ConvBlock2_Pool = base_model.layers[6]
    ConvBlock3_1 = base_model.layers[7]
    ConvBlock3_2 = base_model.layers[8]
    ConvBlock3_3 = base_model.layers[9]
    ConvBlock3_Pool = base_model.layers[10]
    ConvBlock4_1 = base_model.layers[11]
    ConvBlock4_2 = base_model.layers[12]
    ConvBlock4_3 = base_model.layers[13]
    ConvBlock4_Pool = base_model.layers[14]
    ConvBlock5_1 = base_model.layers[15]
    ConvBlock5_2 = base_model.layers[16]
    ConvBlock5_3 = base_model.layers[17]
    ConvBlock5_Pool = base_model.layers[18]

    #NewLayer = ConvBlock5_3(ConvBlock5_2)


    # Reconstructing neural network architecture w/ batch normalization
    x = Input.output
    x = ConvBlock1_1(x)
    x = ConvBlock1_2(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock1_Pool(x)
    x = ConvBlock2_1(x)
    x = ConvBlock2_2(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock2_Pool(x)
    x = ConvBlock3_1(x)
    x = ConvBlock3_2(x)
    x = ConvBlock3_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock3_Pool(x)
    x = ConvBlock4_1(x)
    x = ConvBlock4_2(x)
    x = ConvBlock4_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock4_Pool(x)
    x = ConvBlock5_1(x)
    x = ConvBlock5_2(x)
    x = ConvBlock5_3(x)
    x = GaussianNoise(0.1)(x)
    x = ConvBlock5_Pool(x)

    ### INSERT REST OF LAYERS HERE ###
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    #x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    #x = GaussianNoise(0.1)(x) # INSERT BATCH NORMALIZATION LAYER HERE
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation="softmax")(x)
    model = Model(input = base_model.input, output = predictions)
    sgd = optimizers.SGD(learning_rate=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=["accuracy"])

    # Train model and evaluate performance
    print('We are now training cross-validation set #',i+1)
    Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

    # Display and store performance results
    Results.history['loss'] = [round(l, 4) for l in Results.history['loss']]
    Results.history['accuracy'] = [round(l, 4) for l in Results.history['accuracy']]
    Results.history['val_loss'] = [round(l, 4) for l in Results.history['val_loss']]
    Results.history['val_accuracy'] = [round(l, 4) for l in Results.history['val_accuracy']]

    print('Training Loss:',Results.history['loss'])
    print('Training Accuracy:',Results.history['accuracy'])
    print('Validation Loss:',Results.history['val_loss'])
    print('Validation Accuracy:',Results.history['val_accuracy'])

    TrainLoss.append(Results.history['loss'])
    TrainAcc.append(Results.history['accuracy'])
    TestLoss.append(Results.history['val_loss'])
    TestAcc.append(Results.history['val_accuracy'])
    print('')


    # Predict values for test set
    Probabilities = model.predict(TestImages)

    # Calculate data for ROC curve
    FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
    All_FPR.append(FPR)
    All_TPR.append(TPR)
    All_thresholds.append(thresholds)

# Save and export as CSV files
with open("ConvNoise_TrainLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainLoss)
with open("ConvNoise_TrainAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TrainAcc)
with open("ConvNoise_TestLoss.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestLoss)
with open("ConvNoise_TestAcc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(TestAcc)
with open("ConvNoise_FPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_FPR)
with open("ConvNoise_TPR.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_TPR)
with open("ConvNoise_Thresholds.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(All_thresholds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


We are now training cross-validation set # 1
Train on 22048 samples, validate on 5510 samples
Epoch 1/75
22048/22048 [==============================] - 41s 2ms/step - loss: 0.6940 - accuracy: 0.7773 - val_loss: 0.1991 - val_accuracy: 0.9230
Epoch 2/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.2422 - accuracy: 0.9022 - val_loss: 0.1528 - val_accuracy: 0.9490
Epoch 3/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1938 - accuracy: 0.9276 - val_loss: 0.1408 - val_accuracy: 0.9537
Epoch 4/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1701 - accuracy: 0.9415 - val_loss: 0.1363 - val_accuracy: 0.9557
Epoch 5/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1595 - accuracy: 0.9447 - val_loss: 0.1315 - val_accuracy: 0.9579
Epoch 6/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1490 - accuracy: 0.9479 - val_loss: 0.1281 - val_accuracy: 0.9575
Epoch 7/75
22048/22048 [

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



22048/22048 [==============================] - 40s 2ms/step - loss: 0.1094 - accuracy: 0.9610 - val_loss: 0.1094 - val_accuracy: 0.9646
Epoch 20/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1065 - accuracy: 0.9619 - val_loss: 0.1088 - val_accuracy: 0.9652
Epoch 21/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1069 - accuracy: 0.9625 - val_loss: 0.1079 - val_accuracy: 0.9641
Epoch 22/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1026 - accuracy: 0.9635 - val_loss: 0.1113 - val_accuracy: 0.9642
Epoch 23/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1052 - accuracy: 0.9620 - val_loss: 0.1077 - val_accuracy: 0.9655
Epoch 24/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1039 - accuracy: 0.9623 - val_loss: 0.1068 - val_accuracy: 0.9639
Epoch 25/75
22048/22048 [==============================] - 40s 2ms/step - loss: 0.1000 - accuracy: 0.9628 - val_loss: 0.1074

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Examine RAM Requirements

In [ ]:
# Examine RAM Usage
import sys
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)